<a href="https://colab.research.google.com/github/malinphy/Embedding_calls/blob/main/BERT_MLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#### This notebook was created to test Bert Masked Encoder to be used for Masked language model and the Seq4Rec using MultiHeadAttention Layer
# 80% Replace with a [MASK] token: For 80% of the selected inputs the token is replaced with a [MASK] token similar to the classic cloze tests mentioned earlier.
# 10% Replace with an incorrect word: For 10% of the selected inputs, the token is replaced by another randomly selected word whose only requirement is that it’s different from the selected token.
# 10% Replace with the correct word: The remaining 10% of the time the selected token is simply replaced with the correct token.



In [2]:
import numpy as np
import pandas as pd 
import os 
import sys 
import re, string

import matplotlib.pyplot as plt 
import tensorflow as tf 
from tensorflow import keras 
from tensorflow.keras import layers 
from tensorflow.keras.layers import Dense, TextVectorization, Embedding, MultiHeadAttention, Dropout,LayerNormalization

In [3]:
#### PARAMETER LIST 
SEQUENCE_LEN =  256
EMBED_DIM = 128
CORPUS_SIZE = 20000
N_NEURONS = 128
NUM_HEADS = 8
KEY_DIM = 16
NUM_ATT_LAYER = 8

In [4]:
def custom_standardization(input_data):
    lowercase = str(input_data).lower()
    s = re.sub("<br />", " ",lowercase)
    s = re.sub('\x96|\x85|\xe3',' ',s) 
    out = re.sub('[%s]' % re.escape(string.punctuation), '', s)
    return out

In [5]:
data_url = 'https://raw.githubusercontent.com/malinphy/datasets/main/IMDB_sent/IMDB%20Dataset.csv'
df = pd.read_csv(data_url)

In [6]:
df['review']=df['review'].apply(custom_standardization)

In [7]:
df.head(3)

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production the filming te...,positive
2,i thought this was a wonderful way to spend ti...,positive


In [8]:
vectorizer = TextVectorization(max_tokens = CORPUS_SIZE, output_mode="int",output_sequence_length=256)
vectorizer.adapt(df['review'])
# print(vectorizer(df['review'][1]))
#### corpus is crated to collect all tokens
corpus_1 = vectorizer.get_vocabulary()
tokenizer_size = vectorizer.vocabulary_size()
corpus_2 = corpus_1 + ['MASK']

In [9]:
#### creating a dictionary to encode and decode tokens and index of the tokens
vocab_2enc = {i:j for i,j in enumerate(corpus_1)}
enc_2vocab = {j:i for i,j in enumerate(corpus_1)}
#### subscript_2 refers to masked corpus
vocab_2enc_2 = {i:j for i,j in enumerate(corpus_2)}
enc_2vocab_2 = {j:i for i,j in enumerate(corpus_2)}

In [10]:
def mask_creator(tokenizer_size):
  x = np.random.randint(0,len(tokenizer_size),int(len(tokenizer_size)*0.15  ))
  
  return x

random_indexes = mask_creator(corpus_2)
random_indexes

mask_index = enc_2vocab_2['MASK']
print('Index of [\'MASK\'] token',mask_index)
for i in random_indexes:
  x = vocab_2enc_2[i]
  vocab_2enc_2[i] = 'MASK'
  enc_2vocab_2[x] = mask_index

Index of ['MASK'] token 20000


In [12]:
#### masking the text 
total_review = []
for j in range(len(df)):
  x  = []
  for i in (vectorizer(df['review'][j])):
    x.append(vocab_2enc_2[((i).numpy())])
    big = ((" ".join((x))))
  total_review.append(big)




NameError: ignored

In [13]:
sayac = 0
total_tokens = []
for i in range(len(total_review)):
  # sayac +=1
  # if sayac ==3:
  #   break
  iki = []
  for j in total_review[i].split():
    # print(enc_2vocab_2[j])
    iki.append(enc_2vocab_2[j])
    uc = np.pad(iki, (0, SEQUENCE_LEN - len(iki)), 'constant')
  total_tokens.append(uc)

In [ ]:
# sayac = 0
# total_tokens = []
# for i in range(len(total_review)):
#   # sayac +=1
#   # if sayac ==3:
#   #   break
#   iki = []
#   for j in total_review[i].split():
#     # print(enc_2vocab_2[j])
#     iki.append(enc_2vocab_2[j])
#     uc = np.pad(iki, (0, SEQUENCE_LEN - len(iki)), 'constant')
#   total_tokens.append(uc)

In [ ]:
def get_pos_encoding_matrix(max_len, d_emb):
    pos_enc = np.array(
        [
            [pos / np.power(10000, 2 * (j // 2) / d_emb) for j in range(d_emb)]
            if pos != 0
            else np.zeros(d_emb)
            for pos in range(max_len)
        ]
    )
    pos_enc[1:, 0::2] = np.sin(pos_enc[1:, 0::2])  # dim 2i
    pos_enc[1:, 1::2] = np.cos(pos_enc[1:, 1::2])  # dim 2i+1
    return pos_enc

In [14]:
first_input = tf.keras.Input(shape = (SEQUENCE_LEN,), name= 'word_iputs')
word_embeddings = Embedding(CORPUS_SIZE+1,EMBED_DIM)(first_input)
total_embs = word_embeddings #+ get_pos_encoding_matrix(SEQUENCE_LEN,EMBED_DIM)
mha_1 = MultiHeadAttention(num_heads = NUM_HEADS, key_dim = KEY_DIM, name = 'first_embedding_layer')(total_embs,total_embs,total_embs)
mha_1_out = mha_1+total_embs
norm_1 = LayerNormalization(epsilon=0.001, name= 'normalization_layer_1')(mha_1_out)

#### Feed forward
feed_forward = tf.keras.Sequential([
                                    Dense(N_NEURONS, activation= 'relu'),
                                    Dense(EMBED_DIM)
                                    ])
ff_out = feed_forward(norm_1)
norm_2 = LayerNormalization(epsilon=0.001, name= 'normalization_layer_2')(norm_1 +ff_out)
top_layer = Dense(CORPUS_SIZE+1, activation='relu', name = 'selection_layer')(norm_2)

In [ ]:
# for i in range(NUM_ATT_LAYER):
#   mha_1 = MultiHeadAttention(num_heads = NUM_HEADS, key_dim = KEY_DIM, name = 'first_embedding_layer')(total_embs,total_embs,total_embs)
#   mha_1_out = mha_1+total_embs
#   norm_1 = LayerNormalization(epsilon=0.001, name= 'normalization_layer_1')(mha_1_out)

#   #### Feed forward
#   feed_forward = tf.keras.Sequential([
#                                     Dense(N_NEURONS, activation= 'relu'),
#                                     Dense(16)
#                                     ])
#   ff_out = feed_forward(norm_1)
#   norm_2 = LayerNormalization(epsilon=0.001, name= 'normalization_layer_2')(norm_1 +ff_out)
#   total_embs = norm_2

In [15]:
attention_model = tf.keras.Model(inputs = first_input, outputs = top_layer)

In [ ]:
# attention_model.summary()

In [16]:
attention_model.compile(
    loss = 'SparseCategoricalCrossentropy',
    optimizer = tf.keras.optimizers.Adam(
    learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07),
    metrics = ['accuracy']

)

In [17]:
y = np.array(vectorizer(df['review']))
total_tokens = np.array(total_tokens)

In [19]:
attention_model.fit(((total_tokens)),y, epochs = 10)

Epoch 1/10
313/313 [==============================] - 117s 374ms/step - loss: 9.9350 - accuracy: 0.3077
Epoch 2/10
313/313 [==============================] - 117s 373ms/step - loss: 9.9036 - accuracy: 0.3118
Epoch 3/10
313/313 [==============================] - 117s 373ms/step - loss: 9.9036 - accuracy: 0.3118
Epoch 4/10
313/313 [==============================] - 117s 373ms/step - loss: 9.9036 - accuracy: 0.3118
Epoch 5/10
313/313 [==============================] - 117s 373ms/step - loss: 9.9036 - accuracy: 0.3118
Epoch 6/10
313/313 [==============================] - 117s 373ms/step - loss: 9.9036 - accuracy: 0.3118
Epoch 7/10
313/313 [==============================] - 117s 373ms/step - loss: 9.9036 - accuracy: 0.3118
Epoch 8/10
313/313 [==============================] - 117s 373ms/step - loss: 9.9036 - accuracy: 0.3118
Epoch 9/10
313/313 [==============================] - 117s 372ms/step - loss: 9.9036 - accuracy: 0.3118
Epoch 10/10
313/313 [==============================] - 117s 373m

In [ ]:
len(total_tokens)

In [ ]:
np.array(total_tokens)

In [ ]:
uzunluklar